# Dialogue RPT

In [28]:
import torch

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [3]:
import json
import pandas as pd

In [4]:
model_card = "microsoft/DialogRPT-updown"   
tokenizer = AutoTokenizer.from_pretrained(model_card)
model = AutoModelForSequenceClassification.from_pretrained(model_card)

def score(cxt, hyp):
  model_input = tokenizer.encode(cxt + "<|endoftext|>" + hyp, return_tensors="pt")
  result = model(model_input, return_dict=True)
  return torch.sigmoid(result.logits)

# GPT 3 json file scoring

In [13]:
f_gpt= open('../data-generator/gpt-3/GPT-3_section_level.json')
gpt_3=json.load(f_gpt)

In [14]:
gpt_questions=[]
gpt_answers=[]
gpt_scores=[]

In [15]:
from tqdm.auto import tqdm

In [16]:
length_gpt=len(gpt_3)
length_gpt
for i in tqdm(range(length_gpt)):
    context = gpt_3[i]['positive_ctxs']['text']
    #print(context)
    answer= gpt_3[i]['answers']
    #print(answer)
    try:
        current_score= score(context,answer).item()
    except:
        current_score='error'
    gpt_scores.append(current_score)
    gpt_questions.append(gpt_3[i]['questions'])
    gpt_answers.append(answer)

  0%|          | 0/144 [00:00<?, ?it/s]

In [17]:
df_gpt=pd.DataFrame()
df_gpt['questions']=gpt_questions
df_gpt['answers']=gpt_answers
df_gpt['scores']=gpt_scores
df_gpt.head(15)

,questions,answers,scores
0,\nQ. How does the design process for a digital...,\n\nA. The design process for a digital FSM ty...,0.569915
1,\nQ. Why is it important to design digital sys...,\n\nA. Most problems in developing large digit...,0.520266
2,\nQ. What is a Gray code?,\n\nA Gray code is a cycle over all bit patter...,0.607294
3,\nQ. How does a three-bit gray code counter work?,\n\nA.\n\nA three-bit gray code counter works ...,0.538617
4,\nQ. Is it possible to create a counter with f...,"\n\nA. No, it is not possible.",0.512374
5,\nQ. What is the purpose of the INIT input in ...,\n\nA. The INIT input allows us to force the F...,error
6,\nQ. What is the mechanism for stopping and st...,\n\nA. The mechanism for stopping and starting...,0.501597
7,\nQ. What is the purpose of the two-bit Gray c...,\n\nA. The purpose of the two-bit Gray code is...,0.626356
8,\nQ. Why is it important to specify all possib...,\n\nAns. It is important to specify all possib...,error
9,\nQ. Why is it often a good idea to start by s...,\n\nA. The right strategy is thus often to sta...,error


In [18]:
df_gpt.dtypes

questions    object
answers      object
scores       object
dtype: object

In [19]:
df_gpt=df_gpt[df_gpt['scores']!='error']

In [20]:
df_gpt['scores']=df_gpt['scores'].astype(float)

In [21]:
df_gpt[df_gpt['scores']>0.7]

,questions,answers,scores
22,\nQ. What is an encoder?,\n\nA. An encoder is a component that helps to...,0.722736
104,\nQ. What is the difference between unsigned a...,\n\nA.\n\nThe difference between unsigned and ...,0.701529


In [22]:
df_gpt['scores'].mean()

0.5429410876213819

In [23]:
df_gpt.to_csv('gpt_scoring.csv')